# Keyword Spotting

In this post, you’ll get to explore it for yourself. Now that you have seen how the Keyword Spotting application works we thought it might be fun to explore how well our default pre-trained model works! This will also start to get you familiar with the (relatively large amount of) code we are going to use to train a custom model later as that Colab will build directly on top of this one. And for fun we’ve added in a section where you can upload your own audio samples to see if the model can figure out if you are saying "yes" or "no" or something else "unknown." This is the summary of lecture "Applications of TinyML" from edX.

- toc: true 
- badges: true
- comments: true
- author: Chanseok Kang
- categories: [Python, edX, Deep_Learning, Tensorflow, tinyML]
- image: 

In [1]:
import tensorflow as tf
import sys

sys.path.append('speech_commands')
import input_data
import models
import numpy as np
import pickle

print('Tensorflow v' + tf.__version__)

Tensorflow v2.4.0


This notebook uses the pre-trained [micro_speech](https://github.com/tensorflow/tensorflow/tree/master/tensorflow/lite/micro/examples/micro_speech) example for [TensorFlow Lite for MicroControllers](https://www.tensorflow.org/lite/microcontrollers/overview) 20 kB [Simple Audio Recognition](https://www.tensorflow.org/tutorials/sequences/audio_recognition) model to recognize keywords!

## Configure Defaults
In this Colab we will just run with the default configurations to use the pre-trained model. However, in your assignment you will try the model to recognize a new word.

In [2]:
# A comma-delimited list of the words you want to train for.
# All the other words you do not select will be used to train 
# an "unknown" label so that the model does not just recognize
# speech but your specific words. Audio data with no spoken 
# words will be used to train a "silence" label.
WANTED_WORDS = "yes,no"

# Print the configuration to confirm it
print("Spotting these words: %s" % WANTED_WORDS)

Spotting these words: yes,no


In [3]:
# Calculate the percentage of 'silence' and 'unknown' training samples required
# to ensure that we have equal number of samples for each label.
number_of_labels = WANTED_WORDS.count(',') + 1
number_of_total_labels = number_of_labels + 2 # for 'silence' and 'unknown' label
equal_percentage_of_training_samples = int(100.0/(number_of_total_labels))
SILENT_PERCENTAGE = equal_percentage_of_training_samples
UNKNOWN_PERCENTAGE = equal_percentage_of_training_samples

# Constants which are shared during training and inference
PREPROCESS = 'micro'
WINDOW_STRIDE = 20
MODEL_ARCHITECTURE = 'tiny_conv'

# Constants for training directories and filepaths
DATASET_DIR =  'dataset/'
LOGS_DIR = 'logs/'
TRAIN_DIR = './train/' # for training checkpoints and other files.

# Constants for inference directories and filepaths
import os
MODELS_DIR = 'models'
if not os.path.exists(MODELS_DIR):
    os.mkdir(MODELS_DIR)
MODEL_TF = os.path.join(MODELS_DIR, 'model.pb')
MODEL_TFLITE = os.path.join(MODELS_DIR, 'model.tflite')
FLOAT_MODEL_TFLITE = os.path.join(MODELS_DIR, 'float_model.tflite')
MODEL_TFLITE_MICRO = os.path.join(MODELS_DIR, 'model.cc')
SAVED_MODEL = os.path.join(MODELS_DIR, 'saved_model')

# Constants for Quantization
QUANT_INPUT_MIN = 0.0
QUANT_INPUT_MAX = 26.0
QUANT_INPUT_RANGE = QUANT_INPUT_MAX - QUANT_INPUT_MIN

# Constants for audio process during Quantization and Evaluation
SAMPLE_RATE = 16000
CLIP_DURATION_MS = 1000
WINDOW_SIZE_MS = 30.0
FEATURE_BIN_COUNT = 40
BACKGROUND_FREQUENCY = 0.8
BACKGROUND_VOLUME_RANGE = 0.1
TIME_SHIFT_MS = 100.0

# URL for the dataset and train/val/test split
DATA_URL = 'https://storage.googleapis.com/download.tensorflow.org/data/speech_commands_v0.02.tar.gz'
VALIDATION_PERCENTAGE = 10
TESTING_PERCENTAGE = 10

## Loading the pre-trained model

These commands will download a pre-trained model checkpoint file (the output from training) that we can use to build a model.

In [4]:
TOTAL_STEPS = 15000 # used to identify which checkpoint file

## Generate a TensorFlow Model for Inference

Combine relevant training results (graph, weights, etc) into a single file for inference. This process is known as freezing a model and the resulting model is known as a frozen model/graph, as it cannot be further re-trained after this process.

In [5]:
model_settings = models.prepare_model_settings(
    len(input_data.prepare_words_list(WANTED_WORDS.split(','))),
    SAMPLE_RATE, CLIP_DURATION_MS, WINDOW_SIZE_MS,
    WINDOW_STRIDE, FEATURE_BIN_COUNT, PREPROCESS)
audio_processor = input_data.AudioProcessor(
    DATA_URL, DATASET_DIR,
    SILENT_PERCENTAGE, UNKNOWN_PERCENTAGE,
    WANTED_WORDS.split(','), VALIDATION_PERCENTAGE,
    TESTING_PERCENTAGE, model_settings, LOGS_DIR)

In [15]:
float_converter = tf.lite.TFLiteConverter.from_saved_model(SAVED_MODEL)
float_tflite_model = float_converter.convert()
float_tflite_model_size = open(FLOAT_MODEL_TFLITE, "wb").write(float_tflite_model)
print('Float model is %d bytes' % float_tflite_model_size)

INFO:tensorflow:Restoring parameters from models\saved_model\variables\variables


INFO:tensorflow:Restoring parameters from models\saved_model\variables\variables


INFO:tensorflow:The given SavedModel MetaGraphDef contains SignatureDefs with the following keys: {'serving_default'}


INFO:tensorflow:The given SavedModel MetaGraphDef contains SignatureDefs with the following keys: {'serving_default'}


INFO:tensorflow:input tensors info: 


...

INFO:tensorflow: tensor name: labels_softmax:0, shape: (1, 4), type: DT_FLOAT


INFO:tensorflow: tensor name: labels_softmax:0, shape: (1, 4), type: DT_FLOAT


INFO:tensorflow:Restoring parameters from models\saved_model\variables\variables


INFO:tensorflow:Restoring parameters from models\saved_model\variables\variables


Float model is 68164 bytes


In [16]:
def representative_dataset_gen():
    for i in range(100):
        data, _ = audio_processor.get_data(1, i*1, model_settings,
                                         BACKGROUND_FREQUENCY, 
                                         BACKGROUND_VOLUME_RANGE,
                                         TIME_SHIFT_MS,
                                         'testing',
                                         )
        flattened_data = np.array(data.flatten(), dtype=np.float32).reshape(1, 1960)
        yield [flattened_data]

converter = tf.lite.TFLiteConverter.from_saved_model(SAVED_MODEL)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
tflite_model = converter.convert()
tflite_model_size = open(MODEL_TFLITE, "wb").write(tflite_model)
print("Quantized model is %d bytes" % tflite_model_size)

INFO:tensorflow:Restoring parameters from models\saved_model\variables\variables


INFO:tensorflow:Restoring parameters from models\saved_model\variables\variables


INFO:tensorflow:The given SavedModel MetaGraphDef contains SignatureDefs with the following keys: {'serving_default'}


INFO:tensorflow:The given SavedModel MetaGraphDef contains SignatureDefs with the following keys: {'serving_default'}


INFO:tensorflow:input tensors info: 


...

INFO:tensorflow: tensor name: labels_softmax:0, shape: (1, 4), type: DT_FLOAT


INFO:tensorflow: tensor name: labels_softmax:0, shape: (1, 4), type: DT_FLOAT


INFO:tensorflow:Restoring parameters from models\saved_model\variables\variables


INFO:tensorflow:Restoring parameters from models\saved_model\variables\variables


TypeError: get_data() missing 1 required positional argument: 'sess'